In [1]:
from sklearn.datasets import load_wine

In [2]:
wine = load_wine()

In [3]:
x = wine.data
y = wine.target #멀티클래스

In [4]:
x.shape, y.shape

((178, 13), (178,))

In [5]:
print(wine.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            10.6  30.0    19.5   3.3
    Ma

* 조건
    1. training set / test set / validation set 구분하기!
    2. scaling (min-max, standardization 둘 중!)
    3. y에 대한 전처리
    4. 히든 레이어 최소 2개 쓰기 (노드와 액티베이션 자유!)
    5. early stopping, validation_data)

In [6]:
import tensorflow as tf
from tensorflow import keras

In [7]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.1, random_state = 2023)

In [8]:
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1, random_state = 2023)

In [9]:
train_x.shape, train_y.shape, val_x.shape, val_y.shape

((144, 13), (144,), (16, 13), (16,))

In [ ]:
# #스케일링
# max_n, min_n = train_x.max(), train_x.min()
# train_x = (train_x - min_n) / (max_n - min_n)
# test_x = (test_x - min_n) / (max_n - min_n)
# train_x.max(), train_x.min()

(1.0, 0.0)

In [10]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
mm_scaler, sd_scaler = MinMaxScaler(), StandardScaler()

In [11]:
import pandas as pd
pd.DataFrame(train_x, columns = wine.feature_names).describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
count,144.000000,144.000000,144.000000,144.0000,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000
mean,13.022778,2.332917,2.372500,19.3625,99.902778,2.336806,2.100764,0.356250,1.595139,5.003819,0.957889,2.679097,756.263889
std,0.798531,1.108230,0.269211,3.3394,13.593663,0.638590,0.992632,0.122068,0.551479,2.303946,0.227866,0.715818,322.009316
min,11.450000,0.890000,1.700000,11.2000,70.000000,0.980000,0.340000,0.130000,0.410000,1.740000,0.480000,1.270000,290.000000
25%,12.355000,1.610000,2.227500,16.9500,89.000000,1.772500,1.272500,0.260000,1.257500,3.240000,0.777500,2.057500,500.000000
50%,13.050000,1.845000,2.360000,19.0000,98.000000,2.415000,2.245000,0.340000,1.560000,4.550000,0.960000,2.845000,679.000000
75%,13.695000,3.000000,2.542500,21.0000,107.250000,2.850000,2.892500,0.430000,1.950000,6.025000,1.122500,3.230000,986.250000
max,14.750000,5.800000,3.230000,30.0000,151.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.450000,4.000000,1680.000000


In [12]:
train_x_mm = mm_scaler.fit_transform(train_x)
train_x_sd = sd_scaler.fit_transform(train_x)

In [13]:
pd.DataFrame(train_x_sd, columns = wine.feature_names).describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
count,1.440000e+02,1.440000e+02,1.440000e+02,1.440000e+02,1.440000e+02,1.440000e+02,1.440000e+02,1.440000e+02,1.440000e+02,1.440000e+02,1.440000e+02,1.440000e+02,1.440000e+02
mean,2.146431e-15,3.022274e-16,1.406282e-15,-1.881211e-16,4.749287e-16,-1.094803e-15,2.467162e-17,-8.018277e-17,8.264994e-16,8.465451e-16,-1.005369e-15,-9.806970e-16,-7.401487e-17
std,1.003490e+00,1.003490e+00,1.003490e+00,1.003490e+00,1.003490e+00,1.003490e+00,1.003490e+00,1.003490e+00,1.003490e+00,1.003490e+00,1.003490e+00,1.003490e+00,1.003490e+00
min,-1.976465e+00,-1.306545e+00,-2.506755e+00,-2.452833e+00,-2.207437e+00,-2.132107e+00,-1.780024e+00,-1.859949e+00,-2.156522e+00,-1.421566e+00,-2.104557e+00,-1.975384e+00,-1.453037e+00
25%,-8.391772e-01,-6.545931e-01,-5.404900e-01,-7.249568e-01,-8.048480e-01,-8.867592e-01,-8.373240e-01,-7.912490e-01,-6.143799e-01,-7.682367e-01,-7.944079e-01,-8.714040e-01,-7.986053e-01
50%,3.420939e-02,-4.418032e-01,-4.659396e-02,-1.089313e-01,-1.404639e-01,1.228761e-01,1.458139e-01,-1.335875e-01,-6.393999e-02,-1.976624e-01,9.297043e-03,2.325756e-01,-2.407805e-01
75%,8.447624e-01,6.040366e-01,6.336779e-01,4.920691e-01,5.423753e-01,8.064423e-01,8.003966e-01,6.062817e-01,6.457181e-01,4.447782e-01,7.249247e-01,7.722990e-01,7.167148e-01
max,2.170551e+00,3.139406e+00,3.196346e+00,3.196571e+00,3.772020e+00,2.425002e+00,3.011825e+00,2.497059e+00,3.611725e+00,3.482760e+00,2.167190e+00,1.851746e+00,2.878676e+00


In [14]:
val_x_sd = sd_scaler.transform(val_x)
test_x_sd = sd_scaler.transform(test_x)

In [15]:
#y전처리
import numpy as np
from tensorflow.keras.utils import to_categorical
class_n = len(np.unique(train_y))

In [16]:
train_y = to_categorical(train_y, class_n)
val_y = to_categorical(val_y, class_n)
test_y = to_categorical(test_y, class_n)

In [17]:
train_x.shape, train_y.shape

((144, 13), (144, 3))

In [18]:
#모델링 Functional API

#1. 세션 클리어
keras.backend.clear_session()

#2. 사슬처럼 엮기
il = keras.layers.Input(shape = (13, ))
hl = keras.layers.Dense(64, activation = "relu", name = "hidden1")(il)
hl = keras.layers.Dense(32, activation = "relu", name = "hidden2")(hl)
ol = keras.layers.Dense(3, activation = "softmax", name = "output")(hl)

#3. 모델의 시작/끝 지정
model = keras.models.Model(il, ol)

#4. 컴파일
model.compile(loss = "categorical_crossentropy", metrics = ["accuracy"], optimizer = "adam")

#5. 요약
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13)]              0         
                                                                 
 hidden1 (Dense)             (None, 64)                896       
                                                                 
 hidden2 (Dense)             (None, 32)                2080      
                                                                 
 output (Dense)              (None, 3)                 99        
                                                                 
Total params: 3,075
Trainable params: 3,075
Non-trainable params: 0
_________________________________________________________________


In [19]:
#Early stopping
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = "val_loss", min_delta = 0, patience = 5, verbose = 1, restore_best_weights = True)

In [20]:
#학습
model.fit(train_x_sd, train_y, epochs = 10000, verbose = 1, callbacks = [es], validation_data = (val_x_sd, val_y))
#전처리에서 validation set으로 나누고 스케일링을 했으면 validation_data를 이용한다!!!!!!!!!

Epoch 1/10000
5/5 [==============================] - 6s 59ms/step - loss: 1.2051 - accuracy: 0.3681 - val_loss: 1.4070 - val_accuracy: 0.1250
Epoch 2/10000
5/5 [==============================] - 0s 10ms/step - loss: 1.0027 - accuracy: 0.4306 - val_loss: 1.1827 - val_accuracy: 0.2500
Epoch 3/10000
5/5 [==============================] - 0s 10ms/step - loss: 0.8379 - accuracy: 0.6111 - val_loss: 0.9812 - val_accuracy: 0.5000
Epoch 4/10000
5/5 [==============================] - 0s 10ms/step - loss: 0.6939 - accuracy: 0.8056 - val_loss: 0.8111 - val_accuracy: 0.7500
Epoch 5/10000
5/5 [==============================] - 0s 10ms/step - loss: 0.5765 - accuracy: 0.9167 - val_loss: 0.6660 - val_accuracy: 0.9375
Epoch 6/10000
5/5 [==============================] - 0s 10ms/step - loss: 0.4757 - accuracy: 0.9722 - val_loss: 0.5461 - val_accuracy: 0.9375
Epoch 7/10000
5/5 [==============================] - 0s 10ms/step - loss: 0.3919 - accuracy: 0.9722 - val_loss: 0.4485 - val_accuracy: 0.9375
Epoch 

In [24]:
y_pred = model.predict(test_x_sd)
print(y_pred[:5])
print(test_y[:5])

1/1 [==============================] - 0s 22ms/step
[[1.1747108e-03 1.2683038e-02 9.8614222e-01]
 [9.7168966e-05 9.9989724e-01 5.6021117e-06]
 [7.7522517e-04 2.5748273e-03 9.9664992e-01]
 [1.7903768e-04 9.9974269e-01 7.8308018e-05]
 [9.9771714e-01 2.0121848e-03 2.7068279e-04]]
[[0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]
